In [5]:
import os
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re
import time
import random
import urllib3

# ==== CONFIG ====
START_URL = "https://www.sandiego.gov/city-clerk/officialdocs/municipal-code"
OUT_DIR = "municipal_code_pdfs"
META_FILE = "municipal_code_index.csv"
HEADERS = {"User-Agent": "Mozilla/5.0"}
# =================

os.makedirs(OUT_DIR, exist_ok=True)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ---- Helpers ----
def get_soup(url):
    """Return BeautifulSoup for a URL, retrying on transient failures."""
    for attempt in range(3):
        try:
            r = requests.get(url, headers=HEADERS, timeout=15, verify=False)
            r.raise_for_status()
            return BeautifulSoup(r.text, "html.parser")
        except Exception as e:
            print(f"⚠️ Attempt {attempt+1} failed for {url}: {e}")
            time.sleep(2)
    return None

def safe_name(text):
    """Sanitize folder/file names."""
    return re.sub(r"[^A-Za-z0-9_-]+", "_", text.strip())

def find_next_link(soup, base_url):
    """Find 'Next' pagination link using multiple patterns."""
    next_tag = (
        soup.select_one("li.pager__item--next a")
        or soup.find("a", string=lambda s: s and re.search(r"(next|›|»)", s.strip(), re.I))
        or soup.find("a", rel="next")
    )
    return urljoin(base_url, next_tag["href"]) if next_tag and next_tag.get("href") else None

def download_pdf(url, chapter_name):
    """Download one PDF into OUT_DIR/chapter_name/."""
    folder = os.path.join(OUT_DIR, chapter_name)
    os.makedirs(folder, exist_ok=True)
    try:
        resp = requests.get(url, stream=True, headers=HEADERS, timeout=30, verify=False)
        resp.raise_for_status()
        fname = os.path.basename(urlparse(url).path)
        path = os.path.join(folder, fname)
        
        # Skip existing files
        if os.path.exists(path):
            print(f"⏩ Skipping (already exists): {fname}")
            return path

        with open(path, "wb") as f:
            for chunk in resp.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

        print(f"✅ Downloaded: {chapter_name} → {fname}")
        time.sleep(random.uniform(1, 3))  # polite delay
        return path
    except Exception as e:
        print(f"❌ Failed to download {url}: {e}")
        return None

def find_chapter_links(url):
    """Find all chapter links on the main page."""
    soup = get_soup(url)
    if not soup:
        return []
    content = soup.find("div", class_="region-content") or soup
    links = content.find_all("a", href=True)
    chapters = []
    for a in links:
        text = a.get_text(strip=True)
        href = urljoin(url, a["href"])
        if text.lower().startswith("chapter"):
            chapters.append((text, href))
    return chapters

def find_pdfs_in_chapter(chapter_name, chapter_url):
    """Iterate through all pages of a chapter to collect PDFs."""
    print(f"\n📘 {chapter_name}\n   {chapter_url}")
    page_num = 1
    url = chapter_url
    pdfs = []

    while url:
        print(f"   🔹 Page {page_num}: {url}")
        soup = get_soup(url)
        if not soup:
            print("   ❌ Failed to fetch page.")
            break

        # Gather all PDFs on this page
        for a in soup.find_all("a", href=True):
            href = urljoin(url, a["href"])
            if href.lower().endswith(".pdf"):
                pdf_name = a.get_text(strip=True) or os.path.basename(urlparse(href).path)
                pdfs.append((pdf_name, href))

        # Find next page if any
        next_url = find_next_link(soup, url)
        if next_url and next_url != url:
            page_num += 1
            url = next_url
            sleep_time = random.uniform(1.5, 3.0)
            print(f"   ⏭️ Next page → sleeping {sleep_time:.1f}s...")
            time.sleep(sleep_time)
        else:
            print("   ✅ No next page.")
            break

    print(f"   📄 Found {len(pdfs)} PDFs across {page_num} page(s).")
    return pdfs

# ---- MAIN WORKFLOW ----
chapters = find_chapter_links(START_URL)
print(f"\n🔍 Found {len(chapters)} chapters to process.\n")

metadata = []
total_pdfs = 0

for chapter_name, chapter_url in chapters:
    pdfs = find_pdfs_in_chapter(chapter_name, chapter_url)
    total_pdfs += len(pdfs)
    for pdf_name, pdf_url in pdfs:
        metadata.append({
            "chapter": chapter_name,
            "chapter_url": chapter_url,
            "pdf_name": pdf_name,
            "pdf_url": pdf_url,
        })
        # Uncomment below when ready to download:
        file_path = download_pdf(pdf_url, safe_name(chapter_name))
        metadata[-1]["local_path"] = file_path or ""

# Save metadata CSV (for review before downloads)
if metadata:
    meta_path = os.path.join(OUT_DIR, META_FILE)
    with open(meta_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=metadata[0].keys())
        writer.writeheader()
        writer.writerows(metadata)

print(f"\n🎉 Done scanning!")
print(f"📄 Total PDFs found: {total_pdfs}")
print(f"🗂️ Metadata saved in: {os.path.join(OUT_DIR, META_FILE)}")
print(f"⚙️ Ready to enable downloads when verified.")



🔍 Found 15 chapters to process.


📘 Chapter 01, General Provisions
   https://www.sandiego.gov/city-clerk/officialdocs/municipal-code/chapter-1
   🔹 Page 1: https://www.sandiego.gov/city-clerk/officialdocs/municipal-code/chapter-1
   ✅ No next page.
   📄 Found 20 PDFs across 1 page(s).
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art01Division01.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art01Division02.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art01Division03.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art01Division04.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art02Division01.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art02Division02.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art02Division03.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art02Division04.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art02Division05.pdf
✅ Downloaded: Chapter_01_General_Provisions → Ch01Art02Division06.pdf
✅ Downloaded